In [ ]:
import pandas as pd
import qgrid
import json
import os
import functools
import ipywidgets as widgets
from pathlib import Path
from IPython.display import clear_output, display
from ipyleaflet import Map, Icon, Marker, AntPath, basemaps, basemap_to_tiles, LayerException
from OSGridConverter import grid2latlong

import sys
sys.path.append('..')
from src import sandbox, gpx
from src.settings import PROJECT_PATH, DATASET_PATH

In [ ]:
file_gen = DATASET_PATH.glob("*walks.json")

dfs = [] # an empty list to store the data frames
for file in file_gen:
    
    with open(file) as f:
        
        data = json.load(f)
        dfs.append(pd.DataFrame(data))

df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

display(df)

We have two columns for time as some of the higher graded walks are given in summer conditions. Let's combine those.
Similary, some columns are plurals of others, let's combine those too.

In [ ]:
df["Time"] = df["Time"].fillna(df["Time (summer conditions)"])
df = df.drop(columns="Time (summer conditions)")

for col in df.columns:
    if col + 's' in df.columns:
        df[col] = df[col].fillna(df[col + 's'])
        df = df.drop(columns=col + 's')

display(df)

Let's have a look at walks that include Munro's only. While we're at it, let's look at the really tough one's! We'll also drop columns that have all null values.

In [ ]:
munro_walks = df[(df['Munro'].notnull()) & (df['Grade'].astype(int) >= 3)]
munro_walks = munro_walks.dropna(axis=1, how='all')
display(munro_walks)

In [ ]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(57, 356),
    zoom=6
    )

p = Map(
    basemap=basemap_to_tiles(basemaps.WaymarkedTrails.hiking),
    center=(57, 356),
    zoom=6
    )

#mark = Marker(location=[57, 356], icon=icon)
#m.add_layer(mark)

for p in munro_walks["Start Grid Ref"]:
    loc = grid2latlong(p)
    lat = loc.latitude
    lon = 360 + loc.longitude if loc.longitude < 0 else loc.longitude
    mark = Marker(location=[lat, lon])
    m.add_layer(mark)

m

In [ ]:
df = sandbox.main()

widget = qgrid.show_grid(df)
widget

In [ ]:
def get_lat_lon_bounds(df):
    
    north = max(df["lat"])
    south = min(df["lat"])
    east = max(df["lon"])
    west = min(df["lon"])
    
    return [(south, west), (north, east)]

def handle_click(map_handle, data, **kwargs):
    
    route = gpx.parse(data["GPX"])
    route = gpx.positive_long(route)
    latlon = list(zip(route.lat, route.lon))
    path = AntPath(locations=latlon)
    
    try:
        map_handle.substitute_layer(handle_click.current, path)
    except (AttributeError, LayerException):
        map_handle.add_layer(path)
        
    handle_click.current = path
    
    map_handle.center = kwargs["coordinates"]
    map_handle.zoom = 11
    # map_handle.fit_bounds = get_lat_lon_bounds(route)
   
    display(data)
    

In [ ]:
filtered_df = widget.get_changed_df()

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(57, 356),
    zoom=6
    )

display_max = 100

if len(filtered_df.index) < display_max:
    
    for _, p in filtered_df.iterrows():
        mark = Marker(location=(p["lat"], p["lon"]), draggable=False, title=p["Name"])
        mark.on_click(functools.partial(handle_click, m, p))
        m.add_layer(mark)
        
else:
    print(f"Too many walks to show ({len(filtered_df.index)})! Filter below maximum ({display_max}) ")
    
m
        